<a href="https://colab.research.google.com/github/LeandroGripp/ibd_tp2_analise_pocos/blob/master/Tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import io
import os
import requests
import urllib.request 
import sqlite3
from google.colab import drive

In [43]:
pd.set_option('display.max_columns', 500)

In [44]:
subset = True

# Getting Full Data


*   Connecting with database (conn object)
*   Setting up database with tables specified in db.sql file
*   Importing datasets from folder
*   Creating OriginalDataSet table with all the data
*   To use a subset, set the subset flag to true


In [46]:
# Reading DB definition file

conn = sqlite3.connect('/tmp/consult.db')
cursor = conn.cursor()

db_file_url = "https://raw.githubusercontent.com/LeandroGripp/ibd_tp2_analise_pocos/master/db.sql"

urllib.request.urlretrieve(db_file_url, "/tmp/db.sql")
f = io.open("/tmp/db.sql", 'r', encoding='utf-8')
sql = f.read()
cursor.executescript(sql)

In [51]:
# Getting all the file URLs from github
url = "https://api.github.com/repos/leandrogripp/ibd_tp2_analise_pocos/git/trees/master?recursive=1"
r = requests.get(url)
res = r.json()

ds_files = [file["path"][5:] for file in res["tree"] if file["path"].startswith("data/")]

In [52]:
# Read one file 
base_url = "https://raw.githubusercontent.com/LeandroGripp/ibd_tp2_analise_pocos/master/data"

def read_ds_to_df(file_name):
  ano = int(file_name[:-4].split("_")[0])
  mes = int(file_name[:-4].split("_")[1])
  area = file_name[:-4].split("_")[3]

  skip = 7 if mes<9 else 3

  file_path = f"{base_url}/{file_name}"
  urllib.request.urlretrieve(file_path, f"/tmp/{file_name}")

  df = pd.read_csv('/'.join(('/tmp', file_name)), sep=';', header=None, skiprows=skip)
  df = df if mes<9 else df.drop(47,axis='columns')
  df.columns = [ 
      "estado", "bacia", "nome_poco_anp", "nome_poco_operador", "campo", "operador", "numero_contrato", "periodo", "oleo", "condensado", "petroleo", 
      "gas_natural_associado", "gas_natural_nao_associado", "gas_natural_total", "volume_gas_royalties", "agua", "instalacao_destino", "tipo_instalacao", 
      "tempo_producao", "periodo_da_carga", "corrente", "grau_api", "fp_lc", "fp_lv", "fp_mc", "fp_mv", "fp_pc", "fp_pv", 
      "metano", "etano", "propano", "isobutano", "butano", "isopentano", "npentano", "hexanos", "heptanos", "octanos", "nonanos", "decanos", 
      "undecanos", "oxigenio", "nitrogenio", "gas_carbonico", "densidade_glp_gas", "densidade_glp_liquido", "pcs_gp"
  ]
  df["ano"] = file_name[:-4].split("_")[0]
  df["mes"] = file_name[:-4].split("_")[1]
  df["area"] = file_name[:-4].split("_")[3]

  return df

In [53]:
# conn.execute('DELETE FROM OriginalDataSet;')

if subset:
  df_data = read_ds_to_df(ds_files[0])
  df_data.to_sql(name='OriginalDataSet', con=conn, if_exists='append', index=False)
else:
  for i in range(len(ds_files)):
    df_data = read_ds_to_df(ds_files[i])
    df_data.to_sql(name='OriginalDataSet', con=conn, if_exists='append', index=False)

# Locais

In [54]:
conn.execute('DELETE FROM Locais;')

cmd = """
INSERT INTO
  Locais(estado, bacia, campo)
SELECT DISTINCT
  estado, bacia, campo
FROM 
  OriginalDataSet
"""

conn.execute(cmd)

In [55]:
query = """
SELECT *
FROM 
  Locais

"""

df = pd.read_sql_query(query, conn)
df

,id_local,estado,bacia,campo
0,1,Rio de Janeiro,Santos,ITAPU
1,2,Rio de Janeiro,Santos,ATAPU
2,3,Rio de Janeiro,Campos,VIOLA
3,4,São Paulo,Santos,SAPINHOÁ
4,5,São Paulo,Santos,BAÚNA
...,...,...,...,...
95,96,Sergipe,Sergipe,TARTARUGA
96,97,Rio de Janeiro,Santos,URUGUÁ
97,98,Ceará,Ceará,XARÉU
98,99,Rio de Janeiro,Santos,SURURU


# Pocos - TODO

*   Incompleto: problema do poço em vários campos




In [56]:
# INSERT INTO
#   Pocos(nome_poco_anp, id_local)


cmd = """
SELECT DISTINCT
  O.nome_poco_anp, L.estado, L.bacia, L.campo, L.id_local
FROM 
  OriginalDataSet as O
INNER JOIN 
  Locais as L
ON
  O.campo = L.campo AND O.bacia = L.bacia AND O.estado = L.estado
"""

df = pd.read_sql_query(cmd, conn)
df

# conn.execute(cmd)

,nome_poco_anp,estado,bacia,campo,id_local
0,1-BRSA-1116-RJS,Rio de Janeiro,Santos,ITAPU,1
1,1-BRSA-1146-RJS,Rio de Janeiro,Santos,ATAPU,2
2,1-BRSA-319-RJS,Rio de Janeiro,Campos,VIOLA,3
3,1-BRSA-594-SPS,São Paulo,Santos,SAPINHOÁ,4
4,1-BRSA-607-SPS,São Paulo,Santos,BAÚNA,5
...,...,...,...,...,...
1415,9-RO-17D-RJS,Rio de Janeiro,Campos,RONCADOR,30
1416,9-RO-82B-RJS,Rio de Janeiro,Campos,RONCADOR,30
1417,9-SEP-1-RJS,Rio de Janeiro,Santos,SÉPIA,6
1418,9-TBU-2HA-RJS,Rio de Janeiro,Santos,TAMBAÚ,93
